# Gradient Descent and Linear Regression with PyTorch

## Introduction to Linear Regression

We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall, and humidity (input variables or features) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The _learned_ weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region. 

We'll _train_ our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. Let's begin by importing Numpy and PyTorch.

In [1]:
import numpy as np
import torch

### Training Data

The training data can be replaced using 2 matrices: inputs and targets, each with one row per observation, and one column per variable.

In [2]:
# Input (temp, rainfall, humidity)

inputs = np.array([[73, 67, 43],
                  [91, 88, 64],
                  [87, 145, 58],
                  [102, 43, 37],
                  [69, 96, 70]], dtype = 'float32')

In [3]:
# Targets (apples, oranges)

targets = np.array([[56, 70],
                   [81, 101],
                   [119, 133],
                   [22, 37],
                   [103, 119]], dtype = 'float32')

We've separated the input and target variables because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors.

In [4]:
# Convert inputs and targets into tensors

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 145.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear Regression Model from Scratch

The weights and biases (w11, w12,... w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [5]:
# Weights and biases

w = torch.randn(2, 3, requires_grad = True)   # 2 rows, 3 columns
b = torch.randn(2, requires_grad = True)     # 2 rows, 1 column
print(w)
print(b)

tensor([[ 0.9754,  0.5056,  1.1440],
        [-0.9784,  0.8045, -0.9863]], requires_grad=True)
tensor([ 1.1751, -0.3088], requires_grad=True)



torch.randn creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [6]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.


In [7]:
# Generate predictions

preds = model(inputs)
print(preds)

tensor([[ 155.4446,  -60.2393],
        [ 207.6432,  -81.6679],
        [ 225.6974,  -25.9770],
        [ 164.7312, -102.0039],
        [ 197.0944,  -59.6248]], grad_fn=<AddBackward0>)


In [8]:
# Compare with actual targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see that there's a huge difference between the predictions of our model, and the acutal values of the target variables. Obviously, this is bcz we've initialized our model with random weights and biases, and we can't expect it to *just work*.

## Lost Function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

- Calculate the difference between the two matrices (`preds` and `targets`).
- Square all elements of the difference matrix to remove negative values.
- Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error (MSE)**.

In [9]:
diff = preds - targets
torch.sum(diff * diff) / diff.numel()   # numel() method returns the number of elements in a given tensor

tensor(19337.0449, grad_fn=<DivBackward0>)

**19337.0449** is MSE. Higher the number, higher the loss (information loss).

In [10]:
# MSE loss using function

def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

Let's compute the mean squared error for the current predictions of our model.

In [11]:
# Compute Loss

loss = mse(preds, targets)
print(loss)

tensor(19337.0449, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by about 139.057 (the square root of the loss 19337.0449).* And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the loss because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. weights and biases, bcz they have `requires_grad` set to `True`.

In [12]:
# Compute Gradients

loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the drivative of the loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.

In [13]:
#Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.9754,  0.5056,  1.1440],
        [-0.9784,  0.8045, -0.9863]], requires_grad=True)
tensor([[  9823.5508,   9689.8027,   6087.4785],
        [-13292.9521, -14195.5254,  -8831.7168]])


The first element of `w.grad` is actually the **partial derivative** of `loss` w.r.t. the first element of `w`.

## Adjust weights and biases to reduce the loss

The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t. any individual weight or bias element, it will look like the figure shown below.
A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

If a gradient element is **positive**:

- **increasing** the element's value slightly will **increase** the loss
- **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that weight element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero by calling `.zero_()` method. We need to do this, bcz PyTorch accumulates, gradients i.e. the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [15]:
w, b               #weights and biases remain the same

(tensor([[ 0.9754,  0.5056,  1.1440],
         [-0.9784,  0.8045, -0.9863]], requires_grad=True),
 tensor([ 1.1751, -0.3088], requires_grad=True))

## Train the model using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimation algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t. the weights and biases
4. Adjust the weights by subtracting a samll quantity proportional to the gradient
5. Reset the gradients to zero

Let's implement this algorithm step by step.

In [16]:
# Generate predictions

preds = model(inputs)
print(preds)

tensor([[ 155.4446,  -60.2393],
        [ 207.6432,  -81.6679],
        [ 225.6974,  -25.9770],
        [ 164.7312, -102.0039],
        [ 197.0944,  -59.6248]], grad_fn=<AddBackward0>)


Note that the predictions are same as before, since we haven't made any changes to our model.
The same holds true for the loss and gradients.

In [17]:
# Compute loss

loss = mse(preds, targets)
print(loss)

tensor(19337.0449, grad_fn=<DivBackward0>)


In [18]:
# Compute Gradients

loss.backward()
print(w.grad)
print(b.grad)

tensor([[  9823.5508,   9689.8027,   6087.4785],
        [-13292.9521, -14195.5254,  -8831.7168]])
tensor([ 113.9222, -157.9026])


Finally, we update the weights and biases using the gradients computed above.

In [19]:
# Adjust the weights by subtracting a samll quantity proportional to the gradient

with torch.no_grad():
    w -= (w.grad * (1e-5))
    b -= (b.grad * (1e-5))

In [20]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


A few things to note above:

- We use `torch.no_grad()` to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases.
- We multiply the gradients with a really small number (`10^-5` in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the ***learning rate*** of the algorithm.
- After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

In [21]:
# new weights and biases

w, b

(tensor([[ 0.8771,  0.4087,  1.0831],
         [-0.8455,  0.9465, -0.8980]], requires_grad=True),
 tensor([ 1.1739, -0.3072], requires_grad=True))

With the new weights and biases, the model should have lower loss.

In [22]:
# calculate loss

preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(13128.7734, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [23]:
# Train for 100 epochs

for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= (w.grad * (1e-5))
        b -= (b.grad * (1e-5))
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [24]:
# calculate losss

preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(288.0985, grad_fn=<DivBackward0>)


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [25]:
preds, targets

(tensor([[ 61.6158,  67.3035],
         [ 86.6479,  86.4031],
         [101.7139, 164.8475],
         [ 48.8202,  34.5251],
         [ 93.9498,  97.1647]], grad_fn=<AddBackward0>),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs. 

## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [26]:
import torch.nn as nn

In [27]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We are using 15 training examples to illustrate how to work with large datasets in small batches. 

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [28]:
from torch.utils.data import TensorDataset

In [29]:
# Define dataset

train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [30]:
from torch.utils.data import DataLoader

In [31]:
# Define data loader

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

The data loader is typically used in a for-in loop.

In [32]:
for xb, yb in train_dl:
    print('Batch:')
    print(xb)
    print(yb)
    break       # remove break to see all the batches

Batch:
tensor([[74., 66., 43.],
        [73., 66., 44.],
        [92., 87., 64.],
        [91., 88., 64.],
        [91., 87., 65.]])
tensor([[ 57.,  69.],
        [ 57.,  69.],
        [ 82., 100.],
        [ 81., 101.],
        [ 80., 102.]])


In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [33]:
# Define model

model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0463,  0.2964,  0.1409],
        [-0.2667,  0.4462, -0.0504]], requires_grad=True)
Parameter containing:
tensor([-0.4077,  0.1751], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [34]:
# parameters

list(model.parameters())

[Parameter containing:
 tensor([[-0.0463,  0.2964,  0.1409],
         [-0.2667,  0.4462, -0.0504]], requires_grad=True),
 Parameter containing:
 tensor([-0.4077,  0.1751], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before:

In [35]:
# Generate predictions

preds = model(inputs)
preds

tensor([[22.1288,  8.4357],
        [30.4781, 11.9475],
        [43.4530, 33.8423],
        [12.8268, -9.7051],
        [34.7134, 21.0821],
        [21.7861,  7.7228],
        [30.3226, 11.4509],
        [43.5476, 33.5252],
        [13.1696, -8.9922],
        [34.9005, 21.2984],
        [21.9733,  7.9391],
        [30.1354, 11.2346],
        [43.6086, 34.3389],
        [12.6397, -9.9214],
        [35.0561, 21.7950]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining the loss function manually, we can use the built-in function `mse_loss`.

In [36]:
# Import nn.functional
import torch.nn.functional as F

The `nn.functional` package contains many useful loss functions and several other utilities.

In [37]:
# Define loss function

loss_fn = F.mse_loss

In [38]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(4741.3843, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [39]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [40]:
# utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb, yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            print("Epoch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [41]:
fit(120, model, loss_fn, opt, train_dl)

Epoch [10/120], Loss: 164.0806
Epoch [20/120], Loss: 115.4395
Epoch [30/120], Loss: 60.6877
Epoch [40/120], Loss: 58.1864
Epoch [50/120], Loss: 31.0812
Epoch [60/120], Loss: 64.6950
Epoch [70/120], Loss: 21.4803
Epoch [80/120], Loss: 34.7429
Epoch [90/120], Loss: 38.2245
Epoch [100/120], Loss: 19.1595
Epoch [110/120], Loss: 28.1357
Epoch [120/120], Loss: 19.4876


In [42]:
preds

tensor([[22.1288,  8.4357],
        [30.4781, 11.9475],
        [43.4530, 33.8423],
        [12.8268, -9.7051],
        [34.7134, 21.0821],
        [21.7861,  7.7228],
        [30.3226, 11.4509],
        [43.5476, 33.5252],
        [13.1696, -8.9922],
        [34.9005, 21.2984],
        [21.9733,  7.9391],
        [30.1354, 11.2346],
        [43.6086, 34.3389],
        [12.6397, -9.9214],
        [35.0561, 21.7950]], grad_fn=<AddmmBackward>)

In [43]:
# compare targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [44]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.8357, 67.2779]], grad_fn=<AddmmBackward>)

The predicted yield of apples is 53.3 tons per hectare, and that of oranges is 67.46 tons per hectare.

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)

Keep this picture in mind as you work through the next few tutorials. 

